<a href="https://colab.research.google.com/github/Siu0901/AI_study/blob/main/%EB%AC%B8%EC%84%9C_%EC%9E%84%EB%B2%A0%EB%94%A9%EC%9C%BC%EB%A1%9C_%EA%B8%8D%EC%A0%95_%EB%B6%80%EC%A0%95_%EB%A6%AC%EB%B7%B0_%EA%B0%80%EB%A0%A4%EB%82%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
vocab_size = 20000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)
print('훈련용 리뷰 개수 :',len(X_train))
print('테스트용 리뷰 개수 :',len(X_test))

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000


In [9]:
print('훈련 데이터의 첫번째 샘플 :',X_train[0])
print('훈련 데이터의 첫번째 샘플의 레이블 :',y_train[0])

훈련 데이터의 첫번째 샘플 : [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
훈련 데이터의 첫번째 샘플의 레이블 : 1


In [10]:
print('훈련용 리뷰의 평규 길이: {}'.format(np.mean(list(map(len, X_train)), dtype=int)))
print('테스트용 리뷰의 평균 길이: {}'.format(np.mean(list(map(len, X_test)), dtype=int)))

훈련용 리뷰의 평규 길이: 238
테스트용 리뷰의 평균 길이: 230


In [12]:
# 평균 보다 좀 큰 400으로 패딩
max_len = 400

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
print('X_train의 크기(shape) :', X_train.shape)
print('X_test의 크기(shape) :', X_test.shape)

X_train의 크기(shape) : (25000, 400)
X_test의 크기(shape) : (25000, 400)


In [13]:
# 모델 설계
# GlobalAveragePooling1D()은 입력으로 들어오는 모든 벡터들의
# 평균을 구하는 역할
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 64

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))

# 모든 벡터 평균 구하기
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('embedding_average_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc], validation_split=0.2)

Epoch 1/10
622/625 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.5936 - loss: 0.6718
Epoch 1: val_acc improved from -inf to 0.70120, saving model to embedding_average_model.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - acc: 0.5940 - loss: 0.6716 - val_acc: 0.7012 - val_loss: 0.5456
Epoch 2/10
622/625 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.7898 - loss: 0.4784
Epoch 2: val_acc improved from 0.70120 to 0.85800, saving model to embedding_average_model.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - acc: 0.7899 - loss: 0.4782 - val_acc: 0.8580 - val_loss: 0.3787
Epoch 3/10
624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.8565 - loss: 0.3504
Epoch 3: val_acc improved from 0.85800 to 0.87120, saving model to embedding_average_model.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - acc: 0.8565 - loss: 0.3503 - val_acc: 0.8712 - val_loss: 0.3304
Epoch 4/10
624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.8888 - loss: 0.2877
Epoch 4: val_acc improved from 0.87120 to 0.88260, saving model to embedding_average_model.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - acc: 0.8888 - loss: 0.2877 - val_acc: 0.8826 - val_loss: 0.3004
Epoch 5/10
623/625 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.9065 - loss: 0.2511
Epoch 5: val_acc did not improve from 0.88260
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - acc: 0.9065 - loss: 0.2511 - val_acc: 0.8284 - val_loss: 0.3680
Epoch 6/10
623/625 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.9118 - loss: 0.2269
Epoch 6: val_acc did not improve from 0.88260
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - acc: 0.9119 - loss: 0.2268 - val_acc: 0.8724 - val_loss: 0.3004
Epoch 7/10
624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.9203 - loss: 0.2038
Epoch 7: val_acc improved from 0.88260 to 0.89320, saving model to embedding_average_model.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - acc: 0.9203 - loss: 0.2038 - val_acc: 0.8932 - val_loss: 0.2793
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.9317 - loss: 0.1818
Epoch 8: val_acc did not improve from 0.89320
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - acc: 0.9317 - loss: 0.1818 - val_acc: 0.8928 - val_loss: 0.2777
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.9448 - loss: 0.1597
Epoch 9: val_acc did not improve from 0.89320
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - acc: 0.9448 - loss: 0.1597 - val_acc: 0.8716 - val_loss: 0.3167
Epoch 10/10
622/625 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - acc: 0.9499 - loss: 0.1461
Epoch 10: val_acc did not improve from 0.89320
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.9499 - loss: 0.1461 - val_acc: 0.8730 - val_loss: 0.3194


In [14]:
loaded_model = load_model('embedding_average_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - acc: 0.8849 - loss: 0.2896

 테스트 정확도: 0.8841


In [26]:
import numpy as np

# X_test[0]만 사용하여 예측
# 모델은 여러 샘플을 입력으로 받기 때문에, 단일 샘플도 2D 배열 형태로 맞춰줘야 합니다.
# X_test[0]의 shape은 (max_len,) 이므로, (1, max_len)으로 바꿔줍니다.

single_test_sample = np.expand_dims(X_test[2], axis=0)
# 또는 single_test_sample = X_test[0].reshape(1, -1)

prediction = loaded_model.predict(single_test_sample)
print('예측된 확률:', prediction[0][0])

# 확률을 이진 분류로 변환 (0.5 이상이면 1, 아니면 0)
# binary_prediction = 1 if prediction[0][0] > 0.5 else 0
binary_prediction = prediction[0][0]
print('예측된 레이블 (0: 부정, 1: 긍정):', binary_prediction)
print('실제 레이블:', y_test[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
예측된 확률: 0.43680027
예측된 레이블 (0: 부정, 1: 긍정): 0.43680027
실제 레이블: 0
